
## Objetivo:
Realizar un análisis de sentimientos en los comentarios de los usuarios de yelp y google maps sobre hoteles de los Estados Unidos. 

## Contexto: 
El código se escribe pensando en su ejecución desde una máquina virtual en Google Cloud Platform (GCP) y en sla utilización de la data dentro de GCP.  
Las reseñas con los comentarios sobre los hoteles se encuentran en un archivo Parquet almacenado en Google Cloud Storage, por lo que el código debe contemplar bajar el archivo desde ahí, trabajar con la data, y regresar el archivo a Cloud Storage con la calificación del sentimiento añadida como columna. 

El análisis de sentimientos se realizará utilizando Vader. 

## ¿Qué es Vader?

VADER (Valence Aware Dictionary and sEntiment Reasoner) es una herramienta de análisis de sentimientos basada en léxico y reglas específicamente diseñada para detectar sentimientos en textos de redes sociales, pero también es eficaz en una amplia variedad de textos. Está incluido en la biblioteca de procesamiento del lenguaje natural (NLP) llamada NLTK en Python.

Aquí hay algunas características y capacidades notables de VADER:

Basado en Léxico: VADER tiene un léxico incorporado, donde cada palabra está asociada con una puntuación de sentimiento. Por ejemplo, la palabra "amor" puede tener una puntuación positiva, mientras que "odio" tiene una puntuación negativa.

Consciente del Contexto: VADER es capaz de entender el contexto de ciertas palabras o frases que podrían cambiar el sentimiento. Por ejemplo, reconocerá que "no es bueno" tiene un sentimiento más negativo que simplemente "bueno".

Reconoce Emoticones y Slang: Una característica destacada de VADER es que reconoce emoticones (por ejemplo, :) o :() y jerga de Internet (como "lol" o "meh") que a menudo se utilizan para expresar sentimientos.

Análisis de Intensidad: VADER no solo determina el sentimiento positivo/negativo/neutro, sino que también determina la intensidad del sentimiento. Por lo tanto, puede distinguir entre "bueno" y "muy bueno", otorgando una puntuación más alta a este último.

Puntuaciones Compuestas: Aunque VADER proporciona puntuaciones para sentimientos positivos, negativos y neutros por separado, también ofrece una puntuación compuesta que resume el sentimiento general del texto.

Rápido y Ligero: Como VADER utiliza un léxico y un conjunto de reglas predefinidos (en lugar de modelos basados en aprendizaje automático), es relativamente rápido y no requiere entrenamiento.

Dado que VADER fue diseñado específicamente para textos de redes sociales, es especialmente útil para analizar sentimientos en datos como tweets, comentarios de Facebook y reseñas de productos donde los usuarios podrían usar emoticones y jerga de Internet. Sin embargo, como cualquier herramienta de análisis de sentimientos, no es infalible y su precisión puede variar según el contexto y el tipo de datos.


## Secciones del código:

#### Importaciones de Módulos:

Se importan las bibliotecas necesarias para la ejecución del programa:
pandas: Utilizado para manipular y procesar datos estructurados.
nltk: Biblioteca para procesar el lenguaje natural.
SentimentIntensityAnalyzer de nltk.sentiment.vader: Herramienta específica para el análisis de sentimientos.
storage de google.cloud: Utilizado para interactuar con Google Cloud Storage.

#### Descarga del Lexicón de VADER:

Se descarga el lexicón necesario para que VADER pueda realizar el análisis de sentimientos. Esta descarga se necesita realizar solo una vez.
Inicialización del Analizador de Sentimientos de VADER:

Se inicializa el analizador de sentimientos que posteriormente se usará para calcular el sentimiento de los comentarios.


#### Función get_sentiment_score:

Esta función toma un comentario como entrada.
Devuelve un puntaje de sentimiento compuesto para el comentario. Un puntaje positivo indica un sentimiento positivo y viceversa.


#### Función process_reviews_from_gcs:

Esta función realiza varios pasos:
Configuración del Cliente de GCS: Inicializa un cliente para interactuar con Google Cloud Storage.
Descarga del Archivo Parquet: Descarga el archivo Parquet desde el bucket especificado a un directorio temporal (/tmp/hoteles.parquet) en la instancia de VM.
Lectura del Archivo: Usa pandas para leer el archivo Parquet descargado en un DataFrame.
Análisis de Sentimientos: Aplica la función get_sentiment_score a la columna comentario del DataFrame y almacena los resultados en una nueva columna llamada puntaje_sentimiento.
Guardar DataFrame Modificado: Guarda el DataFrame actualizado (con la nueva columna de puntaje de sentimiento) como un archivo Parquet en el directorio temporal.
Subida del Archivo Modificado: Sube el archivo Parquet modificado de vuelta al bucket, reemplazando el archivo original.


#### Ejecución de la Función:

Se establecen las constantes BUCKET y ARCHIVO para especificar el nombre del bucket y la ruta del archivo Parquet, respectivamente.
Se ejecuta la función process_reviews_from_gcs con los valores especificados para procesar el archivo, realizar el análisis de sentimientos y guardar los cambios en el mismo archivo en el bucket.

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from google.cloud import storage

# Descargar el recurso necesario para VADER (ejecuta solo una vez)
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimientos de VADER
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(comentario):
    return analyzer.polarity_scores(comentario)['compound']

def process_reviews_from_gcs(bucket_name, blob_name):
    # Crear un cliente de GCS
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    # Descargar el archivo del bucket
    blob = bucket.blob(blob_name)
    blob.download_to_filename('/tmp/hoteles.parquet') # Guardar temporalmente en /tmp
    
    # Leer el archivo Parquet con pandas
    df = pd.read_parquet('/tmp/hoteles.parquet')
    
    # Calcular el sentimiento para cada comentario y agregarlo como columna 'puntaje_sentimiento'
    df['puntaje_sentimiento'] = df['comentario'].apply(get_sentiment_score)

    # Guardar el dataframe modificado nuevamente como Parquet
    df.to_parquet('/tmp/hoteles_modified.parquet')
    
    # Subir el archivo Parquet modificado al bucket, reemplazando el original
    blob.upload_from_filename('/tmp/hoteles_modified.parquet')

# Ejecuta la función
BUCKET = 'credenciales_hotel_analytics'
ARCHIVO = 'las_reviews_joi.parquet'
process_reviews_from_gcs(BUCKET, ARCHIVO)